In [1]:
from torchinfo import summary
import torch
from torch import nn

In [14]:
class SimpleCNN1DModel(nn.Module):
    def __init__(self, in_shape, out_shape, hidden_units, signal_length):
        super().__init__()
        self.name = "SimpleCNN1D"
        padding = 1
        kernel_size = 2
        stride = 1
        conv_out_size = int((signal_length+2*padding-kernel_size)/stride + 1)
        self.simple_conv = nn.Sequential(
        nn.Conv1d(in_channels=in_shape, out_channels=hidden_units, kernel_size = kernel_size, stride=stride, padding=padding),
        nn.ReLU(),
        nn.Flatten(),
        nn.Dropout(p=0.5, inplace=False),
        nn.Linear(in_features = hidden_units*conv_out_size, out_features = out_shape)
        )
    def forward(self, x):
        return self.simple_conv(x)

In [15]:
simple_1dcnn = SimpleCNN1DModel(4,6*20,hidden_units=72,signal_length = 36).float()
simple_1dcnn

TypeError: empty(): argument 'size' failed to unpack the object at pos 2 with error "type must be tuple of ints,but got float"

In [6]:
print(summary(model = simple_1dcnn,
        input_size=(1,4,36),
        dtypes=[torch.float],
        verbose=2,
        col_width=16,
        col_names=["kernel_size", "output_size", "num_params", "mult_adds"],
        row_settings=["var_names"],
        ))

RuntimeError: Failed to run torchinfo. See above stack traces for more details. Executed layers up to: [Conv1d: 2, ReLU: 2, Flatten: 2, Dropout: 2]